# Ingest races.csv file

## Pre-requisites
- Add data source and file date parameter widgets to the notebook and get it as a variable
- Run configuration notebook with folder paths
- Run common_functions notebook

In [0]:
dbutils.widgets.text("param_data_source", "")
var_data_source = dbutils.widgets.get("param_data_source")

In [0]:
dbutils.widgets.text("param_file_date", "")
var_file_date = dbutils.widgets.get("param_file_date")

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/common_functions"

### Add ingestion date column to each data frame using current timestamp

### Step 1 - Read the CSV file using the Spark DataFrame Reader

In [0]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DoubleType

In [0]:
races_schema = StructType(fields = [
    StructField("raceId", IntegerType(), False),
    StructField("year", IntegerType(), True),
    StructField("round", IntegerType(), True),
    StructField("circuitId", IntegerType(), False),
    StructField("name", StringType(), True),
    StructField("date", StringType(), True),
    StructField("time", StringType(), True),
    StructField("url", StringType(), True)
])

In [0]:
races_df = spark.read \
    .option("header", True) \
    .schema(races_schema) \
    .csv(f'{raw_folder_path}/{var_file_date}/races.csv')

## Step 2 - Add ingestion date and race timestamp columns

In [0]:
from pyspark.sql.functions import to_timestamp, concat, col, lit

In [0]:
races_with_timestamp_df = races_df.withColumn("race_timestamp", to_timestamp(concat(col("date"), lit(" "), col("time")), "yyyy-MM-dd HH:mm:ss")) \
                                .withColumn("data_source", lit(var_data_source)) \
                                .withColumn("file_date", lit(var_file_date))

In [0]:
races_with_ingestion_date_df = add_ingestion_date(races_with_timestamp_df)

## Step 3 - Select only required columns and rename columns

In [0]:
races_final_df = races_with_ingestion_date_df.select(col("raceId").alias("race_id"), col("year").alias("race_year"), 
                                                col("round"), col("circuitId").alias("circuit_id"), col("name"),
                                                col("ingestion_date"), col("race_timestamp"))

## Step 4 - Write data to Data Lake as a managed table in Delta Lake format

In [0]:
races_final_df.write.mode("overwrite").format("delta").saveAsTable("f1_processed.races")

In [0]:
dbutils.notebook.exit("Success")